In [42]:
## Import Libraries ##
import sqlite3
from sqlite3 import Error
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from pandas import DataFrame
from sqlalchemy import create_engine, MetaData, Table, select, func, desc

## Connect to Database

In [43]:
engine = create_engine('sqlite:///C:/Users/604906/Desktop/BAH/Data Science/WiDS/Wildfires/188-million-us-wildfires/FPA_FOD_20170508.sqlite')

In [44]:
connection = engine.connect()

In [45]:
# Print tables names of database to make sure connection worked
print(engine.table_names())

['ElementaryGeometries', 'Fires', 'KNN', 'NWCG_UnitIDActive_20170109', 'SpatialIndex', 'geometry_columns', 'geometry_columns_auth', 'geometry_columns_field_infos', 'geometry_columns_statistics', 'geometry_columns_time', 'idx_Fires_Shape', 'idx_Fires_Shape_node', 'idx_Fires_Shape_parent', 'idx_Fires_Shape_rowid', 'spatial_ref_sys', 'spatial_ref_sys_aux', 'spatialite_history', 'sql_statements_log', 'sqlite_sequence', 'views_geometry_columns', 'views_geometry_columns_auth', 'views_geometry_columns_field_infos', 'views_geometry_columns_statistics', 'virts_geometry_columns', 'virts_geometry_columns_auth', 'virts_geometry_columns_field_infos', 'virts_geometry_columns_statistics']


## Connect to Fires Table

In [46]:
# Function in SqlAlchemy Library that reflects the table information (such as datatypes, primary keys, etc.)
metadata = MetaData()

In [47]:
# Create a Table object that can be used to query the data
Fires = Table('Fires', metadata, autoload = True, autoload_with=engine)

In [48]:
try:
    stmt = 'SELECT * FROM Fires'
    
    # Execute the statement and store all the records: results
    results = connection.execute(stmt).fetchall()

    # Create a DataFrame from the results: df
    fires = pd.DataFrame(results)

    # Set column names
    fires.columns = results[0].keys()

except Exception as err:
    print(err)

In [49]:
#Look at columns in Fires
#fires.info()

## Pull in Climate Data ##

In [50]:
climate_df = pd.read_csv('climate_df.csv')

In [51]:
climate_df.head(5) #view some data

,state,county,climate,climate_zone,moisture
0,AL,Autauga,Hot-Humid,3,A
1,AL,Baldwin,Hot-Humid,2,A
2,AL,Barbour,Hot-Humid,3,A
3,AL,Bibb,Mixed-Humid,3,A
4,AL,Blount,Mixed-Humid,3,A


In [52]:
#Fix Alaska county names that don't match / Correct county name in climate DF
climate_df['county'] = climate_df['county'].replace('Prince of Wales- Outer Ketchikan', 'Prince of Wales-Outer Ketchikan')
climate_df['county'] = climate_df['county'].replace('Skagway-Hoonah- Angoon', 'Skagway-Hoonah-Angoon')
# Correct county name in fires DF since it isn't accurate
fires['FIPS_NAME'] = fires['FIPS_NAME'].replace('Skagway-Yakutat-Angoon', 'Skagway-Hoonah-Angoon')

In [53]:
#Fix Georgie counties
# Multiple states with same county name
climate_df[climate_df['county'] == 'Bryan']['state'].value_counts()
climate_df[(climate_df['state'] == 'GA') & (climate_df['county'] == 'Bryan')]
# Add row to climate_df to match with Bryan South (GA)
climate_df = climate_df.append({'state':'GA', 'county':'Bryan South', 'climate':'Hot-Humid', 'climate_zone':'2', \
                                'moisture':'A'}, ignore_index=True)

In [54]:
# Multiple states with same county name
climate_df[climate_df['county'] == 'DeKalb']['state'].value_counts()
climate_df[(climate_df['state'] == 'GA') & (climate_df['county'] == 'DeKalb')]
climate_df[(climate_df['state'] == 'TN') & (climate_df['county'] == 'DeKalb')]
# Correct Georgia and Tennessee county names in climate DF
climate_df.loc[428]['county'] = 'Dekalb'
climate_df.loc[2446]['county'] = 'Dekalb'

In [55]:
#Fix Illinois county names
# Correct county name in fires DF since county name isn't correct (for both Illinois and Texas)
fires['FIPS_NAME'] = fires['FIPS_NAME'].replace('De Witt', 'DeWitt')
# Correct county name in fires DF since county name isn't correct
fires['FIPS_NAME'] = fires['FIPS_NAME'].replace('Cumberlin', 'Cumberland')

In [56]:
#Fix Louisiana Counties
# Correct county name in climate DF
climate_df['county'] = climate_df['county'].replace('East Carroll', 'E. Carroll')
# Correct county name in climate DF
climate_df['county'] = climate_df['county'].replace('St. LanB', 'St. Landry')

In [57]:
#Fix Montana and New Mexico Counties
# Correct county name in climate DF
climate_df['county'] = climate_df['county'].replace('Glacier', 'Glacier County')
# Correct county name in climate DF
climate_df['county'] = climate_df['county'].replace('Doña Ana', 'Dona Ana')

In [58]:
#Fix Virginia Counties
climate_df[(climate_df['state'] == 'VA') & (climate_df['county'] == 'Buena Vista')]
# Correct county name in climate DF
climate_df.loc[2836]['county'] = 'Buena Vista City'
climate_df[(climate_df['state'] == 'VA') & (climate_df['county'] == 'Chesapeake')]
# Correct county name in climate DF
climate_df.loc[2843]['county'] = 'Chesapeake City'
climate_df[(climate_df['state'] == 'VA') & (climate_df['county'] == 'Colonial Heights')]
# Correct county name in climate DF
climate_df.loc[2847]['county'] = 'Colonial Heights City'
climate_df[(climate_df['state'] == 'VA') & (climate_df['county'] == 'Hampton')]
# Correct county name in climate DF
climate_df.loc[2875]['county'] = 'Hampton City'
climate_df[(climate_df['state'] == 'VA') & (climate_df['county'] == 'Newport News')]
# Correct county name in climate DF
climate_df.loc[2904]['county'] = 'Newport News City'
climate_df[(climate_df['state'] == 'VA') & (climate_df['county'] == 'Petersburg')]
# Correct county name in climate DF
climate_df.loc[2913]['county'] = 'Petersburg City'
climate_df[(climate_df['state'] == 'VA') & (climate_df['county'] == 'Richmond City')]
# Correct county name in climate DF
climate_df['county'] = climate_df['county'].replace('Richmond City', 'Richmond (city)')
climate_df[(climate_df['state'] == 'VA') & (climate_df['county'] == 'Suffolk')]
# Correct county name in climate DF
climate_df.loc[2939]['county'] = 'Suffolk City'
climate_df[(climate_df['state'] == 'VA') & (climate_df['county'] == 'Virginia Beach')]
# Correct county name in climate DF
climate_df.loc[2943]['county'] = 'Virginia Beach City'
# Correct typo in fires DF
fires['FIPS_NAME'] = fires['FIPS_NAME'].replace('Virginia Beach city', 'Virginia Beach City')

## Check if all counties are aligned ##

In [59]:
#Checked by Tim Balton
# All non-matching counties -- all in Puerto Rico
fires[~fires['FIPS_NAME'].isin(climate_df['county'])].groupby('STATE')['FIPS_NAME'].value_counts().sort_index()

STATE  FIPS_NAME       
PR     Cabo Rojo           46
       Gurabo Municipio     1
       Lajas                8
       Vieques             21
Name: FIPS_NAME, dtype: int64

## Merge Fires and Climate DFs ##

In [60]:
# Merge on state and county
fires = \
fires.merge(climate_df, left_on=['STATE', 'FIPS_NAME'], right_on=['state', 'county'], how='left')

In [20]:
# Drop redundant columns
fires.drop(['state', 'county'], axis=1, inplace=True)
# Make all column names uppercase for consistency
fires = fires.rename(columns={'climate':'CLIMATE', 'climate_zone':'CLIMATE_ZONE', 'moisture':'MOISTURE'})

In [21]:
#Check null data
#fires.isnull().sum()

In [61]:
#Export to CSV (This is the Master DF that is Fires + Climate DFs)
fires.to_csv("test_df.csv")

## United States Population Data

In [4]:
population_df = pd.read_csv('county_population.csv', encoding = "ISO-8859-1")
#reference: 
#population estimates from 1997-2014

In [6]:
#View first five rows
population_df.head(5)

,fips,state_fips,county_fips,areaname,state_name,county_name,fipsst,fipsco,region,division,...,pop2008,pop2009,pop19904,pop20104,base20104,pop2010,pop2011,pop2012,pop2013,pop2014
0,0,0,0,United States,United States,United States,0,0,NaN,NaN,...,NaN,NaN,248790925.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000,1,0,Alabama,Alabama,Alabama,1,0,3.0,6.0,...,4677464.0,4708708.0,4040389.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1001,1,1,Autauga County,Alabama,Autauga County,1,1,3.0,6.0,...,50354.0,50756.0,34222.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1001,1,1001,"Autauga County, Alabama",Alabama,Autauga County,1,1001,NaN,NaN,...,NaN,NaN,NaN,54571.0,54571.0,54684.0,55275.0,55192.0,55136.0,55395.0
4,1003,1,3,Baldwin County,Alabama,Baldwin County,1,3,3.0,6.0,...,176212.0,179878.0,98280.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
#Find unique count of counties in the U.S.
population_df.fips.nunique()
#population_df.state_name.nunique()

3204

In [39]:
population_df.describe()

NameError: name 'population_df' is not defined

In [21]:
population_df2 = pd.read_csv('PopulationEstimates.csv', skiprows=2)
#https://www.ers.usda.gov/data-products/county-level-data-sets/download-data/
#population estimates from 2010-2018

In [38]:
population_df2

NameError: name 'population_df2' is not defined

In [24]:
population_df2.describe()

,FIPS,Rural-urban_Continuum Code_2003,Rural-urban_Continuum Code_2013,Urban_Influence_Code_2003,Urban_Influence_Code_2013,Economic_typology_2015,RESIDUAL_2016,RESIDUAL_2017,RESIDUAL_2018,R_birth_2011,...,R_DOMESTIC_MIG_2017,R_DOMESTIC_MIG_2018,R_NET_MIG_2011,R_NET_MIG_2012,R_NET_MIG_2013,R_NET_MIG_2014,R_NET_MIG_2015,R_NET_MIG_2016,R_NET_MIG_2017,R_NET_MIG_2018
count,3273.000000,3215.000000,3220.000000,3215.000000,3220.000000,3142.000000,3194.000000,3194.000000,3194.000000,3193.000000,...,3193.000000,3193.000000,3193.000000,3193.000000,3193.000000,3193.000000,3193.000000,3193.000000,3193.000000,3193.000000
mean,31358.511763,5.053810,4.937888,5.366096,5.188820,1.808402,0.000000,0.000000,0.000000,11.747667,...,-0.138647,-0.006076,-0.967178,-1.577294,-0.601597,-0.790323,-0.408957,-0.043188,0.987723,1.138240
std,16305.188962,2.701246,2.724344,3.481577,3.506848,1.819511,26.615978,27.820146,33.206809,2.737269,...,12.930164,11.400081,10.446217,11.854181,11.814311,11.150211,12.460067,13.075647,12.874758,11.319971
min,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,-673.000000,-509.000000,-644.000000,0.000000,...,-68.800000,-62.300000,-128.200000,-99.400000,-98.800000,-174.400000,-75.500000,-108.200000,-68.300000,-62.100000
25%,19025.000000,3.000000,2.000000,2.000000,2.000000,0.000000,-2.000000,-2.000000,-2.000000,10.100000,...,-6.500000,-6.100000,-5.800000,-6.900000,-5.900000,-6.200000,-6.000000,-6.000000,-5.300000,-5.000000
50%,30021.000000,6.000000,6.000000,5.000000,5.000000,1.000000,0.000000,0.000000,0.000000,11.500000,...,-0.500000,-0.500000,-0.900000,-1.700000,-1.000000,-1.100000,-1.000000,-0.600000,0.500000,0.600000
75%,46101.000000,7.000000,7.000000,8.000000,8.000000,3.000000,1.000000,1.000000,1.000000,13.000000,...,6.700000,5.900000,3.700000,3.500000,4.400000,4.300000,5.000000,5.900000,7.500000,6.800000
max,72153.000000,9.000000,9.000000,12.000000,12.000000,5.000000,759.000000,718.000000,918.000000,32.200000,...,150.200000,69.400000,122.900000,120.200000,208.300000,150.300000,278.800000,209.700000,150.200000,69.400000


In [25]:
population_df2.FIPS.nunique()

3273

In [52]:
#population_df2.FIPS.isnull().sum()
#population_df2.isnull().sum()
population_df2.isnull()

,FIPS,State,Area_Name,Rural-urban_Continuum Code_2003,Rural-urban_Continuum Code_2013,Urban_Influence_Code_2003,Urban_Influence_Code_2013,Economic_typology_2015,CENSUS_2010_POP,ESTIMATES_BASE_2010,...,R_DOMESTIC_MIG_2017,R_DOMESTIC_MIG_2018,R_NET_MIG_2011,R_NET_MIG_2012,R_NET_MIG_2013,R_NET_MIG_2014,R_NET_MIG_2015,R_NET_MIG_2016,R_NET_MIG_2017,R_NET_MIG_2018
0,False,False,False,True,True,True,True,True,False,False,...,True,True,True,True,True,True,True,True,True,True
1,False,False,False,True,True,True,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3269,False,False,False,False,False,False,False,True,False,False,...,True,True,True,True,True,True,True,True,True,True
3270,False,False,False,False,False,False,False,True,False,False,...,True,True,True,True,True,True,True,True,True,True
3271,False,False,False,False,False,False,False,True,False,False,...,True,True,True,True,True,True,True,True,True,True
3272,False,False,False,False,False,False,False,True,False,False,...,True,True,True,True,True,True,True,True,True,True


In [55]:
population_df2.iloc[[3273]]

,FIPS,State,Area_Name,Rural-urban_Continuum Code_2003,Rural-urban_Continuum Code_2013,Urban_Influence_Code_2003,Urban_Influence_Code_2013,Economic_typology_2015,CENSUS_2010_POP,ESTIMATES_BASE_2010,...,R_DOMESTIC_MIG_2017,R_DOMESTIC_MIG_2018,R_NET_MIG_2011,R_NET_MIG_2012,R_NET_MIG_2013,R_NET_MIG_2014,R_NET_MIG_2015,R_NET_MIG_2016,R_NET_MIG_2017,R_NET_MIG_2018
3273,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [57]:
population_df2.drop(population_df2.index[3273]) #drop that last row with all NAs

,FIPS,State,Area_Name,Rural-urban_Continuum Code_2003,Rural-urban_Continuum Code_2013,Urban_Influence_Code_2003,Urban_Influence_Code_2013,Economic_typology_2015,CENSUS_2010_POP,ESTIMATES_BASE_2010,...,R_DOMESTIC_MIG_2017,R_DOMESTIC_MIG_2018,R_NET_MIG_2011,R_NET_MIG_2012,R_NET_MIG_2013,R_NET_MIG_2014,R_NET_MIG_2015,R_NET_MIG_2016,R_NET_MIG_2017,R_NET_MIG_2018
0,0.0,US,United States,NaN,NaN,NaN,NaN,NaN,"308,745,538","308,758,105",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1000.0,AL,Alabama,NaN,NaN,NaN,NaN,NaN,"4,779,736","4,780,138",...,0.4,1.2,0.5,1.2,1.6,0.6,0.6,0.8,1.1,1.9
2,1001.0,AL,Autauga County,2.0,2.0,2.0,2.0,0.0,"54,571","54,574",...,1.1,0.7,6.0,-6.1,-3.9,2.0,-1.9,5.3,1.0,0.6
3,1003.0,AL,Baldwin County,4.0,3.0,5.0,2.0,5.0,"182,265","182,264",...,22.0,24.3,16.3,17.6,22.9,20.2,17.9,21.5,22.5,24.8
4,1005.0,AL,Barbour County,6.0,6.0,6.0,6.0,3.0,"27,457","27,457",...,-25.5,-9.1,0.3,-6.8,-8.1,-5.1,-15.5,-18.2,-25.0,-8.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3268,72145.0,PR,"Vega Baja Municipio, Puerto Rico",1.0,1.0,1.0,1.0,NaN,"59,662","59,662",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3269,72147.0,PR,"Vieques Municipio, Puerto Rico",7.0,7.0,12.0,12.0,NaN,"9,301","9,301",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3270,72149.0,PR,"Villalba Municipio, Puerto Rico",2.0,2.0,2.0,2.0,NaN,"26,073","26,073",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3271,72151.0,PR,"Yabucoa Municipio, Puerto Rico",1.0,1.0,1.0,1.0,NaN,"37,941","37,941",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## For now just work with Fires.
## Feature Engineering

In [23]:
#fires.info()
fires.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1880465 entries, 0 to 1880464
Data columns (total 42 columns):
OBJECTID                      int64
FOD_ID                        int64
FPA_ID                        object
SOURCE_SYSTEM_TYPE            object
SOURCE_SYSTEM                 object
NWCG_REPORTING_AGENCY         object
NWCG_REPORTING_UNIT_ID        object
NWCG_REPORTING_UNIT_NAME      object
SOURCE_REPORTING_UNIT         object
SOURCE_REPORTING_UNIT_NAME    object
LOCAL_FIRE_REPORT_ID          object
LOCAL_INCIDENT_ID             object
FIRE_CODE                     object
FIRE_NAME                     object
ICS_209_INCIDENT_NUMBER       object
ICS_209_NAME                  object
MTBS_ID                       object
MTBS_FIRE_NAME                object
COMPLEX_NAME                  object
FIRE_YEAR                     int64
DISCOVERY_DATE                float64
DISCOVERY_DOY                 int64
DISCOVERY_TIME                object
STAT_CAUSE_CODE               float64
S

In [26]:
#select columns
df = fires[['DISCOVERY_DATE','DISCOVERY_DOY','DISCOVERY_TIME','CONT_DATE','CONT_DOY','CONT_TIME', #TIME-RELATED
            'STAT_CAUSE_DESCR','FIRE_SIZE','FIRE_SIZE_CLASS', #FIRE-RELATED
            'STATE','FIPS_NAME','FIPS_CODE','LATITUDE','LONGITUDE', #LOCATION-RELATED
            'CLIMATE','CLIMATE_ZONE','MOISTURE']] #WEATHER-RELATED

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1880465 entries, 0 to 1880464
Data columns (total 17 columns):
DISCOVERY_DATE      float64
DISCOVERY_DOY       int64
DISCOVERY_TIME      object
CONT_DATE           float64
CONT_DOY            float64
CONT_TIME           object
STAT_CAUSE_DESCR    object
FIRE_SIZE           float64
FIRE_SIZE_CLASS     object
STATE               object
FIPS_NAME           object
FIPS_CODE           object
LATITUDE            float64
LONGITUDE           float64
CLIMATE             object
CLIMATE_ZONE        object
MOISTURE            object
dtypes: float64(6), int64(1), object(10)
memory usage: 258.2+ MB


In [34]:
df['BURN_TIME'] = df.CONT_DATE - df.DISCOVERY_DATE

C:\Users\604906\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [37]:
df.head(1)
#df_test = pd.read_sql_query("", connection)


,DISCOVERY_DATE,DISCOVERY_DOY,DISCOVERY_TIME,CONT_DATE,CONT_DOY,CONT_TIME,STAT_CAUSE_DESCR,FIRE_SIZE,FIRE_SIZE_CLASS,STATE,FIPS_NAME,FIPS_CODE,LATITUDE,LONGITUDE,CLIMATE,CLIMATE_ZONE,MOISTURE,BURN_TIME
0,2453403.5,33,1300,2453403.5,33.0,1730,Miscellaneous,0.1,A,CA,Plumas,063,40.036944,-121.005833,Cold,5,B,0.0
